In [1]:
import pandas as pd
import numpy as np
import re
import json
import os
from datetime import date


In [2]:
def clean_smb(smb):
    ''' the function does remove the symbols
    in the string data'''

    smb = smb.replace("}",'')
    smb = smb.replace("{",'')
    smb = smb.replace("]",'')
    smb = smb.replace("]",'')
    smb = smb.replace('"','')
    smb = smb.strip('[]')
    smb = smb.split(',')
    return smb

# 1. Data collection

In [3]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

cols = [
    'bodyType', 'brand', 'color','color_hex','complectation_dict',
    'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
    'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
    'name', 'numberOfDoors', 'parsing_unixtime','price', 'priceCurrency',
    'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
    'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
    'Привод', 'Руль', 'Состояние', 'Таможня'
    ]

data = []

json_lst = os.listdir("./data")
filepath = "./data"
for file in json_lst:

    with open(os.path.join(filepath, file), 'r') as json_file:
        json_data  = json.load(json_file)


    for card in json_data:
        data.append(card)

    json_file.close()

train = pd.DataFrame(data,columns=cols)

## General information about dataset
The parsed dataset is actual on 21.08.2021 has been parsed from https://auto.ru/

In [4]:
print(f'The training dataset has been parsed and includes {train.shape[0]} sales offers of cars with {train.shape[1]} features')

The training dataset has been parsed and includes 59762 sales offers of cars with 33 features


In [5]:
train.columns

Index(['bodyType', 'brand', 'color', 'color_hex', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'price', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

As a testing dataset is provided the test.csv

In [6]:
test = pd.read_csv('test.csv')

In [7]:
print(f'The testing dataset includes {test.shape[0]} sales offers of cars with {test.shape[1]} features')


The testing dataset includes 34686 sales offers of cars with 32 features


In [8]:
display(train.sample(3))
display(test.sample(3))


,bodyType,brand,color,color_hex,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
38956,HATCHBACK_3_DOORS,OPEL,,CACECB,{'id': '0'},"Авто в хорошем состоянии , все функционирует и...",1364,140,"{'engine-proof': True, 'cruise-control': True,...",GASOLINE,...,"1.4 MT (140 л.с.), хэтчбек 3 дв., передний, бе...",MECHANICAL,EUROPEAN,2,12,True,FORWARD_CONTROL,LEFT,True,True
56723,SEDAN,VOLKSWAGEN,синий,0000CC,"{'id': '21380914', 'name': 'GT', 'available_op...",Пакеты: \nбезопасность - доп. подушки в кресла...,1395,125,"{'cruise-control': True, 'tinted-glass': True,...",GASOLINE,...,"1.4 AMT (125 л.с.), седан, передний, бензин",ROBOT,EUROPEAN,1,12,True,FORWARD_CONTROL,LEFT,True,True
39811,HATCHBACK_5_DOORS,OPEL,,DEA522,{'id': '0'},"Автомобиль в хорошем состоянии, все электрика ...",1364,90,"{'wheel-heat': True, 'condition': True, 'front...",GASOLINE,...,"1.4 MT (90 л.с.), хэтчбек 5 дв., передний, бензин",MECHANICAL,EUROPEAN,4,12,True,FORWARD_CONTROL,LEFT,True,True


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
28067,седан,MITSUBISHI,https://auto.ru/cars/used/sale/mitsubishi/gala...,серебристый,NaN,"По двигателю нареканий нет, масло менялось каж...",2.4 LTR,150 N12,"{""cruise-control"":true,""tinted-glass"":true,""ai...",бензин,...,SEDAN AUTOMATIC 2.4,автоматическая,JAPANESE,3 или более,8 лет и 10 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен
11196,внедорожник 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/murano/1...,бежевый,NaN,4 эл.стеклоподъемника\nABS\nCD-проигрыватель\n...,3.5 LTR,249 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,ALLROAD_5_DOORS VARIATOR 3.5,вариатор,JAPANESE,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
26499,минивэн,VOLKSWAGEN,https://auto.ru/cars/used/sale/volkswagen/mult...,серебристый,NaN,В рейстайлинг переделывал сам .,2.5 LTR,174 N12,NaN,дизель,...,MINIVAN MECHANICAL 2.5,механическая,EUROPEAN,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59762 entries, 0 to 59761
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              59762 non-null  object
 1   brand                 59762 non-null  object
 2   color                 59762 non-null  object
 3   color_hex             59762 non-null  object
 4   complectation_dict    59762 non-null  object
 5   description           59762 non-null  object
 6   engineDisplacement    59762 non-null  object
 7   enginePower           59762 non-null  object
 8   equipment_dict        59762 non-null  object
 9   fuelType              59762 non-null  object
 10  image                 59762 non-null  object
 11  mileage               59762 non-null  int64 
 12  modelDate             59762 non-null  object
 13  model_info            59762 non-null  object
 14  model_name            59762 non-null  object
 15  name                  59762 non-null

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

## Let's bring the training dataset same to testing dataset.

### Features
 nn |Features | Description | Type
----|---------|-------------|-----
0   |bodyType| Car's body type | Categorical
1   |brand | Car's brand | Categorical
2   |color | Color | Categorical
3   |color_hex  | Color hex code| Categorical
4   |complectation_dic| Dict of complectation | Categorical
5   |description | Description by seller | Categorical
6   |engineDisplacement | Displacement of engine | Quantitative
7   |enginePower | Power of engine | Quantitative
8   |equipment_dict | Dict of equipment | Categorical
9   |fuelType | Type of fuel | Categorical
10  |image | Image of the car | Categorical
11  |mileage | Mileage of the car | Quantitative
12  |modelDate | Year of car's model| Ordinal
13  |model_info | Code of car's model | Categorical
14  |model_name | Name of car's model | Categorical
15  |name | Summary data | Categorical
16  |numberOfDoors | Number of doors | Quantitative
17  |parsing_unixtime | Time of parsing | Ordinal
18  |price | Offered price | Quantitative
19  |priceCurrency | Currency | Quantitative
20  |productionDate | Date of production | Ordinal
21  |sell_id | Seller's ID | Ordinal
22  |super_gen | Dict of features | Categorical
23  |vehicleConfiguration | Dict of options | Categorical
24  |vehicleTransmission | Transmission | Categorical
25  |vendor | Vendor | Categorical
26  |Владельцы | Number of owners | Quantitative
27  |Владение | Holding period month | Quantitative
28  |ПТС | Original vehicle registration | Binary
29  |Привод | Gear type| Categorical
30  |Руль | Steering wheel | Categorical
31  |Состояние | Accident-free car | Binary
32  |Таможня | Custom cleared | Binary

_in training database is no price feature_

## data comparison

In [11]:
def check_df_before_merg(d_df1,d_df2):
    ''' the function to compare of data types both datasets
    was copied from SF colleagues'''

    list_of_names1 = list(d_df1.columns)
    temp_dict = {}
    temp_dict['feature_train'] = list_of_names1
    temp_dict['type_train'] = d_df1.dtypes
    temp_dict['sample_train'] = d_df1.loc[5].values
    temp_df1 = pd.DataFrame.from_dict(temp_dict)


    list_of_names2 = list(d_df2.columns)
    temp_dict2 = {}
    temp_dict2['feature_test'] = list_of_names2
    temp_dict2['type_test'] = d_df2.dtypes
    temp_dict2['sample_test'] = d_df2.loc[5].values
    temp_df2 = pd.DataFrame.from_dict(temp_dict2)

    temp_insert = pd.DataFrame(columns=['< - >'])

    temp_df = pd.concat([temp_df1,temp_insert, temp_df2], axis=1, sort=False)
    temp_df.reset_index(inplace = True)
    del temp_df['index']
    temp_df['< - >'] = '| - |'
    display(temp_df)

    temp_dict3 = {}
    temp_df3= pd.DataFrame(temp_df)
    temp_list  = []
    temp_list2  = []
    temp_list3  = []
    temp_list4  = []


    for i in range(len(temp_df)):
        if str(temp_df3['type_train'][i]) != str(temp_df3['type_test'][i]):
            temp_list.append(temp_df3['feature_train'][i])
            temp_list2.append(temp_df3['feature_test'][i])
            temp_list3.append(str(temp_df3['type_train'][i]) + ' != ' + str(temp_df3['type_test'][i]))
            temp_list4.append(i)

    temp_dict3['index']= temp_list4
    temp_dict3['feature_train']= temp_list
    temp_dict3['divergences'] = temp_list3
    temp_dict3['feature_test']= temp_list2

    temp_df4 = pd.DataFrame.from_dict(temp_dict3)
    temp_df4.set_index('index',inplace=True)

    print(f'The data types are different in {len(temp_df4)} columns\n')
    display(temp_df4)

In [12]:
check_df_before_merg(train, test)

,feature_train,type_train,sample_train,< - >,feature_test,type_test,sample_test
0,bodyType,object,HATCHBACK_5_DOORS,| - |,bodyType,object,лифтбек
1,brand,object,ALFA_ROMEO,| - |,brand,object,SKODA
2,color,object,,| - |,color,object,серый
3,color_hex,object,FAFBFB,| - |,NaN,NaN,NaN
4,complectation_dict,object,{'id': '0'},| - |,complectation_dict,object,"{""id"":""5011515"",""name"":""RS"",""available_options..."
5,description,object,РОЛЬФ Алтуфьево — это целых 30 лет успешной по...,| - |,description,object,Продаю свой автомобиль! Автомобиль полностью и...
6,engineDisplacement,object,1368,| - |,engineDisplacement,object,2.0 LTR
7,enginePower,object,170,| - |,enginePower,object,200 N12
8,equipment_dict,object,"{'cruise-control': True, 'asr': True, 'esp': T...",| - |,equipment_dict,object,"{""cruise-control"":true,""tinted-glass"":true,""es..."
9,fuelType,object,GASOLINE,| - |,fuelType,object,бензин


The data types are different in 8 columns



,feature_train,divergences,feature_test
index,,,
3,color_hex,object != nan,NaN
12,modelDate,object != int64,modelDate
16,numberOfDoors,object != int64,numberOfDoors
18,price,object != nan,NaN
21,sell_id,object != int64,sell_id
31,Состояние,bool != object,Состояние
32,Таможня,bool != object,Таможня
33,NaN,nan != object,car_url


## 1.1 Features treatment before combing of the datasets

### For further processing, training and test datasets must be unified.

### bodyType

#### translation of the feature to 'russian'

In [13]:
#df.bodyType.value_counts(sort=True).keys()
body_dict = {
'SEDAN' :'седан',
'ALLROAD_5_DOORS' :'внедорожник 5 дв.',
'HATCHBACK_5_DOORS' :'хэтчбек 5 дв.',
'WAGON_5_DOORS' :'универсал 5 дв.',
'LIFTBACK' :'лифтбек',
'HATCHBACK_3_DOORS' :'хэтчбек 3 дв.',
'MINIVAN' :'минивэн',
'COMPACTVAN' : 'компактвэн',
'COUPE' : 'купе',
'ALLROAD_3_DOORS' : 'внедорожник 3 дв.',
'COUPE_HARDTOP' : 'купе-хардтоп',
'VAN' : 'фургон',
'PICKUP_TWO' : 'пикап двойная кабина',
'CABRIO' : 'кабриолет',
'ROADSTER' : 'родстер',
'MICROVAN' : 'микровэн',
'ALLROAD_OPEN' : 'внедорожник открытый',
'SEDAN_HARDTOP' : 'седан-хардтоп',
'TARGA' : 'тарга',
'PICKUP_ONE' : 'пикап одинарная кабина',
'PICKUP_ONE_HALF' : 'пикап полуторная кабина',
'HATCHBACK_4_DOORS' : 'хэтчбек 4 дв.',
'LIMOUSINE' : 'лимузин',
'SEDAN_2_DOORS' : 'седан 2 дв.'
}
train['bodyType'] = train['bodyType'].map(body_dict)

In [14]:
train['bodyType'].value_counts(sort=True)


седан                      20686
внедорожник 5 дв.          19682
хэтчбек 5 дв.               7093
универсал 5 дв.             2980
лифтбек                     2455
хэтчбек 3 дв.               1503
минивэн                     1345
компактвэн                  1230
купе                         998
внедорожник 3 дв.            666
фургон                       355
пикап двойная кабина         271
кабриолет                    162
родстер                      121
купе-хардтоп                 111
микровэн                      34
внедорожник открытый          20
седан-хардтоп                 12
пикап одинарная кабина        10
пикап полуторная кабина        9
тарга                          8
хэтчбек 4 дв.                  6
лимузин                        2
седан 2 дв.                    2
Name: bodyType, dtype: int64

### color_hex to color

In [15]:
print('train dataset')
train['color_hex'].value_counts()

train dataset


040001    15571
FAFBFB    11411
97948F     8116
CACECB     6558
0000CC     5914
EE1D19     3199
200204     2505
007F00     1958
C49648     1255
22A0F8      848
FFD600      630
660099      569
4A2197      452
DEA522      420
FF8649      337
FFC0CB       19
Name: color_hex, dtype: int64

In [16]:
#color_dictionary initialization
keys = train['color_hex'].value_counts(dropna=False).keys().tolist()
values = test['color'].value_counts(dropna=False).keys().tolist()
color_dict = dict(zip(keys, values))

#Changing the hex codes to related colors
train['color'] = train['color_hex'].map(color_dict)

In [17]:
#Removing the color_hex feature
train.drop(['color_hex'], axis=1, inplace=True)
train['color'].value_counts()


чёрный         15571
белый          11411
серебристый     8116
серый           6558
синий           5914
красный         3199
коричневый      2505
зелёный         1958
бежевый         1255
голубой          848
золотистый       630
пурпурный        569
фиолетовый       452
жёлтый           420
оранжевый        337
розовый           19
Name: color, dtype: int64

### engineDisplacement of training dataset to same data format

In [18]:
def float_value(feature):
    pattern = re.compile('([^\d.]+)')
    feature = feature.replace(pattern, '')
    oft_value = feature.value_counts(sort=True).keys()[0]
    feature = feature.replace('', oft_value).astype(float, errors = 'raise')
    return feature

In [19]:
print('train dataset')
train['engineDisplacement'] = round(float_value(train['engineDisplacement'])/1000,1)
train['engineDisplacement'].value_counts(sort=True)

train dataset


1.6    14393
2.0    11872
3.0     4557
1.4     3918
1.8     3910
       ...  
6.6        2
8.1        2
6.3        1
6.5        1
7.0        1
Name: engineDisplacement, Length: 65, dtype: int64

In [20]:
print('test dataset')
test['engineDisplacement'] = float_value(test['engineDisplacement'])
test['engineDisplacement'].value_counts(sort=True)

test dataset


2.0    7446
1.6    4973
3.0    3915
1.8    3349
2.5    2477
2.4    1745
3.5    1710
1.4    1635
1.5     725
1.2     485
4.4     447
2.8     384
4.7     377
1.3     365
5.5     361
4.0     356
3.2     348
4.5     346
2.1     271
3.7     242
2.2     236
4.2     228
1.9     221
2.3     203
2.7     189
3.6     183
5.6     170
5.0     158
5.7     135
2.9     131
4.6     108
3.1      88
4.8      84
1.7      59
3.8      57
1.0      57
2.6      55
6.0      53
0.7      53
3.3      44
3.4      36
4.1      35
4.3      32
5.4      31
6.2      28
5.9      12
5.2      12
6.3       8
5.8       7
6.6       6
1.1       4
4.9       4
3.9       1
5.3       1
Name: engineDisplacement, dtype: int64

### enginePower from training dataset to same data format

In [21]:
print('train dataset')
#train['enginePower'].value_counts(sort=True)
train['enginePower'] = float_value(train['enginePower']).astype(int)
train['enginePower'].value_counts(sort=True)

train dataset


150    4541
249    2236
140    1777
105    1613
123    1473
       ... 
714       1
457       1
489       1
700       1
311       1
Name: enginePower, Length: 395, dtype: int64

In [22]:
print('test dataset')
#test['enginePower'].value_counts(sort=True)
test['enginePower'] = round(float_value(test['enginePower'])/100,0).astype(int)
test['enginePower'].value_counts(sort=True)

test dataset


249    1708
150    1699
110    1039
170     977
105     955
       ... 
338       1
402       1
626       1
630       1
639       1
Name: enginePower, Length: 315, dtype: int64

### fuelType from training dataset to same language

In [23]:
#fuelType_dictionary initialization
fuelType_dict = {}
keys = train['fuelType'].value_counts(dropna=False).keys().tolist()
values = test['fuelType'].value_counts(dropna=False).keys().tolist()
fuelType_dict = dict(zip(keys, values))

train['fuelType'] = train['fuelType'].map(fuelType_dict)

In [24]:
print('test dataset')
test['fuelType'].value_counts()

test dataset


бензин     28601
дизель      5800
гибрид       223
электро       55
газ            7
Name: fuelType, dtype: int64

In [25]:
print('train dataset')
train['fuelType'].value_counts()

train dataset


бензин     50893
дизель      8452
гибрид       244
электро      164
газ            8
Name: fuelType, dtype: int64

### modelDate
_from object to int dtype_

In [26]:
train['modelDate'] = float_value(train['modelDate']).astype('int64')

In [27]:
print('train dataset')
train['modelDate'].dtype

train dataset


dtype('int64')

In [28]:
print('test dataset')
test['modelDate'].dtype

test dataset


dtype('int64')

### numberOfDoors
_from object to int dtype_

In [29]:
print('train dataset')
train['numberOfDoors'] = float_value(train['numberOfDoors']).astype('int64')
train['numberOfDoors'].dtype

train dataset


dtype('int64')

In [30]:
print('test dataset')
test['numberOfDoors'].dtype

test dataset


dtype('int64')

### vehicleTransmission
_from training dataset to same language_

In [31]:
#vehicleTransmission_dictionary initialization
vehicleTransmission_dict = {}
keys = train['vehicleTransmission'].value_counts(dropna=False).keys().tolist()
values = test['vehicleTransmission'].value_counts(dropna=False).keys().tolist()
vehicleTransmission_dict = dict(zip(keys, values))

train['vehicleTransmission'] = train['vehicleTransmission'].map(vehicleTransmission_dict)

In [32]:
print('test dataset')
test['vehicleTransmission'].value_counts(sort=True)

test dataset


автоматическая      19596
механическая         7209
вариатор             3999
роботизированная     3882
Name: vehicleTransmission, dtype: int64

In [33]:
print('train dataset')
train['vehicleTransmission'].value_counts(sort=True)

train dataset


автоматическая      30905
механическая        19672
вариатор             5342
роботизированная     3842
Name: vehicleTransmission, dtype: int64

### Владельцы
_from object to int dtype_

In [34]:
print('train dataset')
train['Владельцы'] = float_value(train['Владельцы']).astype(int)
train['Владельцы'].value_counts(sort=True)

train dataset


1    18079
2    15611
3    14383
4    11689
Name: Владельцы, dtype: int64

In [35]:
print('test dataset')
test['Владельцы'] = float_value(train['Владельцы']).astype(int)
test['Владельцы'].value_counts(sort=True)

test dataset


1    10428
2     9138
3     8319
4     6801
Name: Владельцы, dtype: int64

### ПТС
_from training dataset to same language_

In [36]:
print('test dataset')
orig = test['ПТС'].value_counts(sort=True).keys()[0]
test['ПТС'].value_counts(sort=True)

test dataset


Оригинал    30098
Дубликат     4587
Name: ПТС, dtype: int64

In [37]:
print('train dataset')
orig = train['ПТС'].value_counts(sort=True).keys()[0]
train['ПТС'] = train['ПТС'].map(lambda x: 'Оригинал' if x == orig else 'Дубликат')
train['ПТС'].value_counts(sort=True)

train dataset


Оригинал    51514
Дубликат     8248
Name: ПТС, dtype: int64

### Привод
_from training dataset to same language_

In [38]:
print('test dataset')
test['Привод'].value_counts(sort=True)

test dataset


передний    15565
полный      15020
задний       4101
Name: Привод, dtype: int64

In [39]:
print('train dataset')
train['Привод'].value_counts(sort=True)

#drive_type_dictionary initialization
drive_type_dict = {}
keys = train['Привод'].value_counts(dropna=False).keys().tolist()
values = test['Привод'].value_counts(dropna=False).keys().tolist()
drive_type_dict = dict(zip(keys, values))

train['Привод'] = train['Привод'].map(drive_type_dict)
train['Привод'].value_counts(sort=True)

train dataset


передний    33191
полный      21982
задний       4588
Name: Привод, dtype: int64

### Руль
_from training dataset to same language_

In [40]:
print('test dataset')
test['Руль'].value_counts(sort=True)

test dataset


Левый     33287
Правый     1399
Name: Руль, dtype: int64

In [41]:

print('train dataset')
train['Руль'].value_counts(sort=True)


train dataset


LEFT     58417
RIGHT     1345
Name: Руль, dtype: int64

In [42]:
left = train['Руль'].value_counts(sort=True).keys()[0]

In [43]:
train['Руль'] = train['Руль'].map(lambda x: 'Левый' if x == left else 'Правый')
train['Руль'].value_counts(sort=True)

Левый     58417
Правый     1345
Name: Руль, dtype: int64

### Состояние
_from training dataset to same language_

In [44]:
print('test dataset')
test['Состояние'].value_counts()

test dataset


Не требует ремонта    34686
Name: Состояние, dtype: int64

In [45]:
print('train dataset')
train['Состояние'].value_counts()

train dataset


True    59762
Name: Состояние, dtype: int64

In [46]:
train['Состояние'] = test['Состояние'].value_counts().keys()[0]
train['Состояние'].value_counts()

Не требует ремонта    59762
Name: Состояние, dtype: int64

### Таможня
_from training dataset to same language_

In [47]:
print('test dataset')
test['Таможня'].value_counts()

test dataset


Растаможен    34686
Name: Таможня, dtype: int64

In [48]:
print('train dataset')
train['Таможня'].value_counts()

train dataset


True    59762
Name: Таможня, dtype: int64

In [49]:
train['Таможня'] = test['Таможня'].value_counts().keys()[0]
train['Таможня'].value_counts()

Растаможен    59762
Name: Таможня, dtype: int64

### price is the target feature

In [50]:
train['price'] = float_value(train['price']).astype(int)
#y_train = train['price']
#train = train.drop(['price'], axis = 1)

### car_url
the car_url feature for further ML modeling doesn't matter and has to be removed anyway.

In [51]:
test.drop(['car_url'], axis=1, inplace=True)

### sell_id
_it doesn't bring any useful information. That's why it has be removed from both datasets._

In [52]:
test.drop(['sell_id'], axis=1, inplace=True)
train.drop(['sell_id'], axis=1, inplace=True)

In [1095]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59762 entries, 0 to 59761
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              59761 non-null  object 
 1   brand                 59762 non-null  object 
 2   color                 59762 non-null  object 
 3   complectation_dict    59762 non-null  object 
 4   description           59762 non-null  object 
 5   engineDisplacement    59762 non-null  float64
 6   enginePower           59762 non-null  int32  
 7   equipment_dict        59762 non-null  object 
 8   fuelType              59761 non-null  object 
 9   image                 59762 non-null  object 
 10  mileage               59762 non-null  int64  
 11  modelDate             59762 non-null  int64  
 12  model_info            59762 non-null  object 
 13  model_name            59762 non-null  object 
 14  name                  59762 non-null  object 
 15  numberOfDoors      

In [53]:
#check
check_df_before_merg(train, test)

,feature_train,type_train,sample_train,< - >,feature_test,type_test,sample_test
0,bodyType,object,хэтчбек 5 дв.,| - |,bodyType,object,лифтбек
1,brand,object,ALFA_ROMEO,| - |,brand,object,SKODA
2,color,object,белый,| - |,color,object,серый
3,complectation_dict,object,{'id': '0'},| - |,complectation_dict,object,"{""id"":""5011515"",""name"":""RS"",""available_options..."
4,description,object,РОЛЬФ Алтуфьево — это целых 30 лет успешной по...,| - |,description,object,Продаю свой автомобиль! Автомобиль полностью и...
5,engineDisplacement,float64,1.4,| - |,engineDisplacement,float64,2.0
6,enginePower,int32,170,| - |,enginePower,int32,200
7,equipment_dict,object,"{'cruise-control': True, 'asr': True, 'esp': T...",| - |,equipment_dict,object,"{""cruise-control"":true,""tinted-glass"":true,""es..."
8,fuelType,object,бензин,| - |,fuelType,object,бензин
9,image,object,//avatars.mds.yandex.net/get-autoru-vos/392674...,| - |,image,object,https://avatars.mds.yandex.net/get-autoru-vos/...


The data types are different in 1 columns



,feature_train,divergences,feature_test
index,,,
17,price,int32 != nan,NaN


## _Now, the training and testing datasets have been brought to same format_

## save to csv file

In [54]:
#save the dataframe to file

today = date.today()
fielname = f'{today}_train.csv'
train.to_csv(fielname, encoding='utf-8')